In [15]:
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head() # 데이터 프레임의 첫번째 행부터 5개 추출

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [16]:
# pandas unique : 명시된 컬럼에 중복 제거
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [17]:
# Species : 타겟 데이터(정답 데이터), 그외 - 학습 데이터
import numpy as np
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [18]:
fish_target = fish['Species'].to_numpy() # 타겟 데이터(정답 데이터)

In [23]:
from sklearn.model_selection import train_test_split #학습 세트, 테스트 세트 분리

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

In [24]:
# 데이터 전처리 - 표준 점수 변환
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input) # 훈련 세트 훈련 데이터를 표준 점수로 변환
test_scaled = ss.transform(test_input) # 테스트 세트 데이터를 표준 점수로 변환

In [25]:
# k-최근접 이웃 분류 - 분류, 확률 예측
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors = 3)
kn.fit(train_scaled, train_target) # 학습

print(kn.score(train_scaled, train_target)) # 룬련 세트 정확도
print(kn.score(test_scaled, test_target)) # 테스트 세트 정확도

0.8907563025210085
0.85


In [26]:
# 클래스 목록 (분류 항목) - 알파벳 순서대로
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [27]:
print(kn.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


In [29]:
# 분류 기준 확률 kn.predict_proba(..)
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.    0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    1.    0.   ]
 [0.    0.    0.    1.    0.    0.    0.   ]
 [0.    0.    0.667 0.    0.333 0.    0.   ]
 [0.    0.    0.667 0.    0.333 0.    0.   ]]


In [31]:
distances, indexes = kn.kneighbors(test_scaled[2:3])
print(train_target[indexes]) # 확률로 보기에는 조금 어색 -> 로지스틱 회귀

[['Pike' 'Pike' 'Pike']]


In [33]:
# 로지스틱 회귀 - 이진 분류(도미, 방어)
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]
print(target_bream_smelt[:5])

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [34]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt) # 훈련

LogisticRegression()

In [36]:
print(lr.predict(train_bream_smelt[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [38]:
# 확률 predict_proba
print(np.round(lr.predict_proba(train_bream_smelt[:5]), decimals=3))

[[0.998 0.002]
 [0.027 0.973]
 [0.995 0.005]
 [0.986 0.014]
 [0.998 0.002]]


In [39]:
print(lr.classes_) #1 - 양성 클래스 : 방어, 2 - 음성 클래스 - 방어

['Bream' 'Smelt']


In [40]:
# 가중치(기울기)와 절편
print(lr.coef_, lr.intercept_)

[[-0.40451732 -0.57582787 -0.66248158 -1.01329614 -0.73123131]] [-2.16172774]


In [41]:
# 학습을 통해서 구한 가중치와 절편으로 방정식의 결과 값을 구하는 함수 decision_function
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions) # 결과값을 0~1 사이로 변환 - 시그모이드 함수

[-6.02991358  3.57043428 -5.26630496 -4.24382314 -6.06135688]


In [43]:
# 시그모이드 함수로 값을 0~1 사이로 변환
from scipy.special import expit # 시그모이드 함수
print(np.round(expit(decisions), decimals=3))

[0.002 0.973 0.005 0.014 0.002]


In [48]:
# 로지스틱 회귀 - 다중 분류 / 확률 - 소프트맥스 함수
lr = LogisticRegression(C= 20, max_iter=1000)
lr.fit(train_scaled, train_target) # 학습

print(lr.score(train_scaled, train_target)) # 학습 세트
print(lr.score(test_scaled, test_target)) # 테스트 세트

0.9327731092436975
0.925


In [49]:
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']


In [50]:
print(lr.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [52]:
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.    0.014 0.842 0.    0.135 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.934 0.015 0.016 0.   ]
 [0.011 0.034 0.305 0.006 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


In [53]:
print(lr.coef_, lr.intercept_)

[[-1.50605457 -1.03747914  2.60919712  7.69422745 -1.18603344]
 [ 0.19200043 -1.99988813 -3.79617723  6.5031264  -2.00022751]
 [ 3.55793539  6.36988929 -8.52233733 -5.75397233  3.79233437]
 [-0.11453312  3.61060122  3.94464502 -3.62243905 -1.75981677]
 [-1.40843721 -6.09242192  5.28629634 -0.86696571  1.84518458]
 [-1.33419936  1.4815339   1.38217545 -5.6602348  -4.39228967]
 [ 0.61328843 -2.33223522 -0.90379937  1.70625805  3.70084845]] [-0.10345052 -0.2728122   3.24444853 -0.17565068  2.64960024 -6.72042842
  1.37829305]


In [54]:
# decision_function
decisions = lr.decision_function(test_scaled[:5])
print(np.round(decisions, decimals=3))

[[ -6.507   1.039   5.168  -2.757   3.337   0.346  -0.625]
 [-10.879   1.944   4.776  -2.423   2.987   7.842  -4.247]
 [ -4.338  -6.237   3.167   6.481   2.364   2.434  -3.872]
 [ -0.691   0.446   2.643  -1.215   3.261  -5.701   1.257]
 [ -6.401  -1.991   5.815  -0.135   3.499  -0.088  -0.699]]


In [55]:
from scipy.special import softmax

proba = softmax(decisions, axis = 1) # 행별로 (axis=1)
print(np.round(proba, decimals=3))

[[0.    0.014 0.842 0.    0.135 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.934 0.015 0.016 0.   ]
 [0.011 0.034 0.305 0.006 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
